### Partie 1 Initialisation

In [1]:
import os
os.environ["HF_HOME"] = "./.cache"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["HF_TRUST_REMOTE_CODE"] = "1"
os.environ["LC_ALL"] = "en_US.UTF-8"
os.environ["LANG"] = "en_US.UTF-8"


Necessaire pour activer la flash attention des A100

In [2]:
!pip install evaluate transformers accelerate sentencepiece rouge-score sacrebleu bitsandbytes imblearn triton tiktoken
!pip install flash-attn --no-build-isolation

import pandas as pd
import csv
import re
import time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from evaluate import load
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl siz

Necessité d'être sur google collab sur des GPU A100

In [31]:
import pandas as pd
import csv


"""
chargement des données puis filtrage sur les champs Body, From, To, Subject.
filtrage sur les lignes ayant un body > 60 caracteres pour pourvoir faire de la NER.
elimination des lignes ayant des valeurs vide ou espace dans les champs From, To, Subject.
selection de 100 emails au hasard.

"""

data_path = "/content/data/enron_data_fraud_labeled.csv"
emails = pd.read_csv(data_path, quoting=csv.QUOTE_NONE, on_bad_lines="skip")

emails = emails.dropna(subset=['Body', 'From', 'To', 'Subject'])

emails = emails[emails['Body'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 60)]

emails = emails[
    emails['From'].apply(lambda x: isinstance(x, str) and x.strip() != "") &
    emails['To'].apply(lambda x: isinstance(x, str) and x.strip() != "") &
    emails['Subject'].apply(lambda x: isinstance(x, str) and x.strip() != "")
]


if len(emails) >= 100:
    emails = emails.sample(n=100, random_state=42)
else:
    print(f"{len(emails)} mail dispo")

print(emails[['From', 'To', 'Subject', 'Body']].head())


<ipython-input-31-063715d4d8ff>:6: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  emails = pd.read_csv(data_path, quoting=csv.QUOTE_NONE, on_bad_lines="skip")


Nombre total d'emails valides (Body > 40 caractères, 'from', 'to' et 'subject' non vides): 6582
                                                                                                                                                        From  \
maildir sanders-r  <11298271.1075858674982.JavaMail.evans@thyme> "Thu 16 Aug 2001 07:12:20 -0700 (PDT)" gfergus@brobeck.com                             7bit   
                   <11234191.1075859456324.JavaMail.evans@thyme> "Tue 11 Dec 2001 13:39:49 -0800 (PST)" no.address@enron.com                            7bit   
        griffith-j <19385950.1075840576198.JavaMail.evans@thyme> "Thu 25 Oct 2001 05:59:09 -0700 (PDT)" karim@karimrashid.com                           7bit   
        lay-k      <23173152.1075860842076.JavaMail.evans@thyme> "Thu 3 Jan 2002 06:35:02 -0800 (PST)"  danesmommy0702@msn.com  text/plain; charset=us-ascii   
        may-l      <21344927.1075852949324.JavaMail.evans@thyme> "Mon 22 Oct 2001 04:59:03 -0700 (PDT)" 

In [32]:
"""
pretraitement des body

"""
def pretraitement(text):
    if isinstance(text, str):
        text = re.sub(r'\n+', ' ', text)
        text = re.sub(r'\?+', ' ', text)
        text = re.sub(r'[^a-zA-Z0-9@._,!? ]', '', text)
        text = re.sub(r'http\S+', '', text)
        return text.strip()
    else:
        return ""

emails['clean_body'] = emails['Body'].apply(pretraitement)

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoConfig
import torch

"""
chargement des modeles en 8bit et en float16 pour quantiser, temps d'attente bien moins long
faire yes pour chaque demande pendant le deroulé du code, (parfois le trust_remote_code=True ne fonctionne pas)
"""

models = {
    "Qwen2.5-3B": "Qwen/Qwen2.5-3B",
    "Phi-3-small-128k-instruct": "microsoft/Phi-3-small-128k-instruct",
    "TinyLlama-1.1B-Chat": "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
}

pipelines_classification = {}

for model_name, model_path in models.items():

    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

    config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)

    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            config=config,
            device_map="auto",
            torch_dtype=torch.float16,
            load_in_8bit=True,
            trust_remote_code=True
        )
    except ValueError as e:
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            config=config,
            device_map="auto",
            torch_dtype=torch.float32,
            trust_remote_code=True
        )

    classifier = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto",
        torch_dtype=torch.float16,
        batch_size=4,
        trust_remote_code=True
    )

    pipelines_classification[model_name] = classifier


Chargement du modèle Qwen2.5-3B...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Tous les modèles ont été chargés avec succès.


In [33]:
import json
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import defaultdict

def extract_entities(pipeline, text):
    """
    recup des champs 'from', 'to' et 'subject' du body de chaque mail via le prompt suivant, on se sert de la generation de texte.
    """
    prompt = (
        f"Extract the 'from', 'to', and 'subject' fields from the following email body. "
        f"Provide the extracted information in JSON format with keys 'from', 'to', and 'subject'.\n\n"
        f"Email Body: {text}\nExtracted Information:"
    )
    try:
        result = pipeline(prompt, max_new_tokens=150, do_sample=False)
        extracted_text = result[0]['generated_text'].strip()

        #on trouve et parse le json
        json_start = extracted_text.find("{")
        json_end = extracted_text.rfind("}") + 1
        if json_start != -1 and json_end != -1:
            json_text = extracted_text[json_start:json_end]
            #print(json_test)
            try:
                parsed_json = json.loads(json_text)
                return parsed_json
            except json.JSONDecodeError:
                print("Erreur de parsing JSON.")
                return {"error": "Invalid JSON format", "raw_output": extracted_text}
        else:
            return {"error": "No JSON detected", "raw_output": extracted_text}

    except Exception as e:
        print(f"Erreur lors de l'extraction : {e}")
        return {"error": "Extraction error", "exception": str(e)}

def extract_vrai_entities(row):
    """
    recuperation des champs 'from', 'to', 'subject' pour un email donné (du dataset de base).
    c'est pour ensuite comparer les entités extraites avec celles de référence.
    """
    vrai = {
        "from": set(),
        "to": set(),
        "subject": set()
    }

    # on extrait from
    from_field = row.get('from', "")
    if pd.notna(from_field):
        vrai["from"].add(from_field.strip())

    # on extrait to
    to_field = row.get('to', "")
    if pd.notna(to_field):
        # si to a plusieurs entrées
        many_to = [p.strip() for p in to_field.split(",")]
        vrai["to"].update(many_to)

    # on extrait subject
    subject = row.get('subject', "")
    if pd.notna(subject):
        vrai["subject"].add(subject.strip())

    return vrai

# dictionnaire pour stocker les entitées extraites
extracted_entities = {model_name: [] for model_name in pipelines_classification.keys()}
#print(extracted_entities)

# dictionnaire pour stocker les entitées de references
vrai_entities = []

# extraction des entités pour chaque email
for i, (row_index, row) in enumerate(emails.iterrows(), start=1):
    text = row['clean_body']
    if not text or text.strip() == "" or text.lower() == "false":
        for model_name in pipelines_classification.keys():
            extracted_entities[model_name].append({"error": "Empty/invalid"})
        # ajout d'entités de référence vides
        vrai_entities.append({
            "from": set(),
            "to": set(),
            "subject": set()
        })
        continue
    print(f"\n---- Email {i} ----\n{text}\n")

    # extraction des entités de ref
    v_entities = extract_vrai_entities(row)
    vrai_entities.append(v_entities)

    for model_name, classifier in pipelines_classification.items():
        extracted = extract_entities(classifier, text)
        extracted_entities[model_name].append(extracted)


---- Email 1 ----
FW EL0095 J. Birchmans order Here is the California Schedule. Thanks Gary To gfergus@brobeck.com


Modèle : Qwen2.5-3B
Entités extraites :
{
  "from": "gfergus@brobeck.com",
  "to": "gfergus@brobeck.com",
  "subject": "FW EL0095 J. Birchmans order Here is the California Schedule. Thanks Gary"
}

Modèle : Phi-3-small-128k-instruct
Entités extraites :
{
  "from": "Gary",
  "to": "gfergus@brobeck.com",
  "subject": "FW EL0095 J. Birchmans order"
}

Modèle : TinyLlama-1.1B-Chat
Erreur de parsing JSON, extraction brute.
Entités extraites :
{
  "error": "Invalid JSON format",
  "raw_output": "Extract the 'from', 'to', and 'subject' fields from the following email body. Provide the extracted information in JSON format with keys 'from', 'to', and 'subject'.\n\nEmail Body: FW EL0095 J. Birchmans order Here is the California Schedule. Thanks Gary To gfergus@brobeck.com\nExtracted Information:\n\n```\n{\n  \"from\": \"J. Birchmans\",\n  \"to\": \"gary.fergus@brobeck.com\",\n  \

TypeError: 'NoneType' object is not iterable

In [34]:
print(extracted_entities)

{'Qwen2.5-3B': [{'from': 'gfergus@brobeck.com', 'to': 'gfergus@brobeck.com', 'subject': 'FW EL0095 J. Birchmans order Here is the California Schedule. Thanks Gary'}, {'from': 'mailtogeorgia.fogo@enron.com', 'to': 'Downtown Club', 'subject': 'Member Benefit - Career Crisis Workshop'}, {'from': 'DJKREEMY on wheels of Plastic', 'to': 'office@karimrashid.com', 'subject': 'distribution or copying of this email message is strictly prohibited. If you receive this email message in error please notify us immediately by telephone on 212 929 8657 or by email at office@karimrashid.com Thank you.'}, {'from': 'Previous Enron Employee', 'to': 'Enron Kids', 'subject': 'Returning my son to Enron Kids'}, {'from': 'employees or affiliates', 'to': '2001 Carr Futures', 'subject': 'The charts are now available on the web by clicking on the hot links contained in this email. If for any reason you are unable to receive the charts via the web'}, {'from': 'Ronald Carroll', 'to': 'Mary Hain', 'subject': 'Joining

In [89]:
for model, extracts in extracted_entities.items():
    if len(extracts) != len(emails):
        print("non")
    else:
        print("oui")

Modèle 'Qwen2.5-3B' a correctement 100 extraits.
Modèle 'Phi-3-small-128k-instruct' a correctement 100 extraits.
Modèle 'TinyLlama-1.1B-Chat' a correctement 100 extraits.


In [93]:
for i in range(100):
  email_index = i

  # vrai
  v_from = set([emails.iloc[email_index]['From'].strip().lower()]) if pd.notna(emails.iloc[email_index]['From']) else set()
  v_to = set([addr.strip().lower() for addr in emails.iloc[email_index]['To'].split(',')]) if pd.notna(emails.iloc[email_index]['To']) else set()
  v_subject = set([emails.iloc[email_index]['Subject'].strip().lower()]) if pd.notna(emails.iloc[email_index]['Subject']) else set()
  v = {'From': v_from, 'To': v_to, 'Subject': v_subject}

  # entités extraites
  extracted = extracted_sets[email_index]

  print("vraies entitées", v)
  print("extraits:", extracted)


vraies entitées {'From': {'7bit'}, 'To': {'"""fergus'}, 'Subject': {'gary s."" <gfergus@brobeck.com>@enron <imceanotes-+22fergus+2c+20gary+20s+2e+22+20+3cgfergus+40brobeck+2ecom+3e+40enron@enron.com>"'}}
extraits: {'From': set(), 'To': set(), 'Subject': set()}
vraies entitées {'From': {'7bit'}, 'To': {'wellness@enron'}, 'Subject': {'enron houston@enron <??senron houston@enron>'}}
extraits: {'From': {'mailtogeorgia.fogo@enron.com'}, 'To': {'mailtogeorgia.fogo@enron.com'}, 'Subject': {'Please email your cancellation notice to mailtogeorgia.fogo@enron.com as soon as possible'}}
vraies entitées {'From': {'7bit'}, 'To': {'karimrashid <karim@karimrashid.com>@enron'}, 'Subject': {'everybody <karim@karimrashid.com>'}}
extraits: {'From': set(), 'To': set(), 'Subject': set()}
vraies entitées {'From': {'text/plain; charset=us-ascii'}, 'To': {'7bit'}, 'Subject': {'"""michael laird"" <danesmommy0702@msn.com>@enron"'}}
extraits: {'From': set(), 'To': set(), 'Subject': set()}
vraies entitées {'From':